In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import math

## First approach: we use NLTK library to tokenize a list of questions to binary vectors

In [2]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gabedarley/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/gabedarley/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
def read_file(answer, question):
    answers, questions = [], []
    
    with open(answer, 'r') as file:
        answers = file.readlines()
        
    with open(question, 'r') as file:
        questions = file.readlines()
        
    return answers, questions    

In [4]:
!pwd
!ls

/Users/gabedarley/Desktop/cb/notebooks
NLTK-TFIDF.ipynb SQLite.ipynb     answers.txt      questions.txt


In [15]:
answers, questions = read_file("answers.txt", "questions.txt")
len(answers), len(questions)

(50, 50)

In [16]:
sw = stopwords.words("english")

def find_similarity(questions, user):
    ranks = []    
    
    # tokenize the user's question
    tokenized_user = word_tokenize(user)
    user_set = {w for w in tokenized_user if not w in sw}
    
    for idx, question in enumerate(questions):
        l1, l2 = [], []
        
        # tokenize the DB's question
        tokenized_question = word_tokenize(question)
        question_set = {w for w in tokenized_question if not w in sw}
        
        rvector = question_set.union(user_set)
        for w in rvector:
            if w in user_set: l1.append(1)
            else: l1.append(0)
                
            if w in question_set: l2.append(1)
            else: l2.append(0)
                
        c = 0
        # cosine formular
        for i in range(len(rvector)):
            c += l1[i]*l2[i]
        cosine = c / ((sum(l1)**0.5*(sum(l2))**0.5))
        
        # for each question, find its similarity to user's question
        ranks.append((idx, cosine))
        
    # sort the ranks
    ranks.sort(key=lambda y: y[1], reverse=True)
    return ranks

def answer(ranks, answers):
    f_idx, s_idx = ranks[0][0], ranks[1][0]
    
    print("The first answer is", answers[f_idx])
    print("The second answer is", answers[s_idx])

In [17]:
ranks = find_similarity(questions, "Can you tell me what an IEP is")
answer(ranks, answers)

The first answer is In making changes to a student's IEP after the annual IEP Team meeting for a school year, the parent of a student with a disability and the public agency may agree not to convene an IEP Team meeting for the purposes of making those changes, and instead may amend or modify the student's current IEP.

The second answer is The LDE shall ensure the following. 1.	Each public agency shall take steps, including the provision of supplementary aids and services determined appropriate and necessary by the student's IEP Team, to provide nonacademic and extracurricular services and activities in the manner necessary to afford students with disabilities an equal opportunity for participation in those services and activities. 2.	Nonacademic and extracurricular services and activities may include counseling services, athletics, transportation, health services, recreational activities, special interest groups or clubs sponsored by the public agency, referrals to agencies that provi

## Second approach: instead of using NLTK, we use TFIDF from scikit-learn library

In [18]:
# TFIDF stands for frequency-inverse document frequency
# try to find the most frequent and significant words

# tf-idf = term_frequency * inverse_document_frequency
# inverse_document_frequency = log(total number of documents / number of documents with term) + 1
# Ex: a word that appears a lot in 1-2 pages is significant

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial.distance import cosine
from pathlib import Path
import glob

In [93]:
directory_path = "../data"
text_files = glob.glob(f"{directory_path}/*.txt")
titles = [Path(text_file).stem for text_file in text_files]
text_files, titles
print(questions)
print(answers)

['What is an LEA?\n', 'What is IDEA?\n', 'What is FAPE?\n', 'What is an IEP?\n', 'What is LRE?\n', 'What is PBIS?\n', 'What is a FBA?\n', 'What is a BIP?\n', 'What is a MDR?\n', 'What is Child Find?\n', 'Should I have my child evaluated for special education?\n', 'What is RTI?\n', 'What is the School Building Level Committee?\n', 'Can the school district refuse to evaluate my child until he or she goes through the RTI process?\n', 'Can the school use my private evaluation to expedite the evaluation process?\n', 'Is the public school system required to evaluate my child that attends a private school for free?\n', 'What does 1508 evaluation mean?\n', 'What are the exceptionalities?\n', 'How long does it take to get my child evaluated?\n', 'Can the school ask for evaluation extensions?\n', 'What happens if the parent disagree with the evaluation?\n', 'If my child is eligible for special education services, how long will it take to start the services?\n', 'Will I receive a notice about the

In [21]:
tfidf_vectorizer = TfidfVectorizer(input='filename', stop_words="english")
tfidf_vector = tfidf_vectorizer.fit_transform(text_files)

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [ ]:
tfidf_df = pd.DataFrame(tfidf_vector.toarray(), index=titles, columns=tfidf_vectorizer.get_feature_names())
tfidf_df.head()

In [ ]:
tfidf_df.loc['doc_freq'] = (tfidf_df > 0).sum()
df = tfidf_df.T.sort_values(by=['questions', 'answers'], ascending=False)
df

### Analysis
Although TFIDF in this case is a good approach, it may not help us solve the problem of ranking questions with user's question in term of similarity. TFIDF will output the significance of a term based on all question/answer pairs. However, these pairs are independent and unrelated. Furthermore, a user's questiion and DB's question both have significant words but they are unrelated, so their similarity is incorrect.

### Tasks
1. Form a list of 21 pairs (answer+question)
2. Fit and transform TF-IDF vectorizer for these 21 pairs, then turn each pair(string) into a vector.
3. For any new sentence (user's input), transform it into a vector of the same dimension.
4. Use cosine similarity to rank user's input vs a list of defined questions

In [ ]:
data = [answer + " " + question for question, answer in zip(questions, answers)]
len(data), data[:1]

In [ ]:
tfidf_vectorizer = TfidfVectorizer(input='content', stop_words="english")
tfidf_vector = tfidf_vectorizer.fit_transform(data)

print(tfidf_vector.shape)
tfidf_vector.toarray()

In [ ]:
def find_similarity_tfidf(questions_vector, user, vectorizer):
    user_vector = vectorizer.transform([user]).toarray().flatten()

    ranks = []
    for idx in range(len(questions_vector)):
        question_vector = questions_vector[idx, :].flatten()

        # cosine formular
        c = 1.0 - cosine(question_vector, user_vector)
        ranks.append((idx, c))

    sorted_ranks = sorted(ranks, key=lambda x: x[1], reverse=True)
    return sorted_ranks

ranks = find_similarity_tfidf(tfidf_vector.toarray(), "Can you please tell me what IEP is?", tfidf_vectorizer)

answer(ranks, answers)

In [35]:
data = [
'What happens if my child’s teacher misses an IEP meeting?',
'Is it okay for a team member to miss a meeting?',
'What should I do if a team member is late to the meeting?',
'My child’s teacher arrived late to our IEP meeting. Is that OK?',
'What are the rules regarding missing an IEP meeting?'
]

tfidf_vectorizer = TfidfVectorizer(input='content', stop_words="english")
tfidf_vector = tfidf_vectorizer.fit_transform(data)

user = 'Can IEP members miss the IEP meeting, come in late, or leave early?'
find_similarity_tfidf(tfidf_vector.toarray(), user, tfidf_vectorizer)

[(3, 0.4305602175990155),
 (1, 0.3408731755908132),
 (2, 0.3103269467449228),
 (4, 0.3059733778253322),
 (0, 0.29413468614195415)]

## Third approach: sentence similarity with Spacy using pre-trained model

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_md

In [ ]:
import spacy
nlp = spacy.load("en_core_web_md")

In [ ]:
def find_similarity_spacy(questions, user, model):
    user_doc = model(user)
    
    ranks = []
    for idx, question in enumerate(questions):
        question_doc = model(question)
        similarity = user_doc.similarity(question_doc)
        ranks.append((idx, similarity))
    
    sorted_ranks = sorted(ranks, key=lambda x: x[1], reverse=True)
    return sorted_ranks

ranks = find_similarity_spacy(questions, "Can you please tell me what IEP is?", nlp)
answer(ranks, answers)

## Fourth approach: sentence similarity with sentence transformers (the baseline is BERT model)

Resources:
1. https://github.com/UKPLab/sentence-transformers
2. https://www.sbert.net/docs/usage/semantic_textual_similarity.html

In [27]:
!pip install -U sentence-transformers

     |████████████████████████████████| 78 kB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 3.5 MB 11.1 MB/s eta 0:00:01
     |████████████████████████████████| 3.7 MB 45.1 MB/s eta 0:00:01
     |████████████████████████████████| 147.2 MB 51 kB/s s eta 0:00:01    |██████████████▉                 | 68.4 MB 5.5 MB/s eta 0:00:15     |█████████████████▎              | 79.2 MB 5.5 MB/s eta 0:00:13
     |████████████████████████████████| 1.2 MB 40.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 20.9 MB/s eta 0:00:01
     |████████████████████████████████| 67 kB 10.7 MB/s eta 0:00:01
     |████████████████████████████████| 895 kB 6.8 MB/s eta 0:00:01     |██████████████████████▎         | 624 kB 6.8 MB/s eta 0:00:01
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=120999 sha256=afb46ffbfeb360c14f3e35804d7be995f95cfa006b727b87d94735a04d187024
  Stored in directory: /Users/gabedarley/Library/Caches/pip/wheels/

In [28]:
from scipy.spatial.distance import cosine

In [29]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

In [30]:
def find_similarity_transformer(questions, user, model):
    sentence_embeddings = model.encode(questions)
    user_embedding = model.encode(user)

    ranks = []
    for idx, embedding in enumerate(sentence_embeddings):
        c = 1.0 - cosine(user_embedding, embedding)
        ranks.append((idx, c))
        
    return sorted(ranks, key=lambda x: x[1], reverse=True)

In [92]:
user_question = "What are the rules regarding missing an IEP meeting?"
ranks = find_similarity_transformer(questions, user_question, model)

for i in range(len(ranks)):
        if ranks[i][0] == 29:
            correct_index = i

answer(ranks, answers)
print(ranks[correct_index][1])

The first answer is In making changes to a student's IEP after the annual IEP Team meeting for a school year, the parent of a student with a disability and the public agency may agree not to convene an IEP Team meeting for the purposes of making those changes, and instead may amend or modify the student's current IEP.

The second answer is Parents may disagree with all or some parts of the program, placement, or related services proposals. The LEA and the parents should make conciliatory attempts to resolve the disputes, including making modifications to the proposed program, placement, and related services. A LEA may not use a parent's refusal to consent to one service or activity to deny the parent or student any other service, benefit, or activity of the LEA.

0.7735286951065063
